# Analyzing Results with Zeno

This notebook shows how to use the Zeno library to analyze the results from our experiments.

In [1]:
import pandas as pd
import tarfile
import tempfile
import os
import csv
import evaluate
import math
from datasets import load_dataset

from zeno import MetricReturn, ZenoOptions, metric, distill, DistillReturn
from zeno_build.evaluation.text_features.length import input_length, output_length
from zeno_build.evaluation.text_metrics.critique import (
    avg_bert_score,
    bert_score,
)
from zeno_build.experiments.experiment_run import ExperimentRun
from zeno_build.reporting.visualize import visualize


/Users/gneubig/anaconda3/envs/zeno-build/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trg_langs_with_gpt4_results = ["lvs_Latn", "tpi_Latn", "ukr_Cyrl", "lim_Latn", "kat_Geor", "tam_Taml", "mag_Deva", "hau_Latn", "fra_Latn", "acm_Arab", "ssw_Latn", "kmr_Latn", "war_Latn", "ajp_Arab", "pbt_Arab", "gle_Latn", "ron_Latn", "sna_Latn", "ckb_Arab", "ibo_Latn"]
trg_langs = trg_langs_with_gpt4_results

In [3]:
# read `data/flores200_dataset.tar.gz`
# Create a temporary directory
with tempfile.TemporaryDirectory() as tmpdirname:
    # Extract the archive contents to the temp directory
    with tarfile.open("data/flores200_dataset.tar.gz", "r:gz") as tar:
        tar.extractall(tmpdirname)
    # Read the contents of English
    with open(os.path.join(tmpdirname, "flores200_dataset", "devtest", "eng_Latn.devtest"), "r") as f:
        src_data = [x.strip() for x in f.readlines()] * len(trg_langs)
    # Read the contents of the target language
    trg_data = []
    trg_lang_data = []
    for trg_lang in trg_langs:
        with open(os.path.join(tmpdirname, "flores200_dataset", "devtest", f"{trg_lang}.devtest"), "r") as f:
            trg_data.extend([x.strip() for x in f.readlines()])
        trg_lang_data.extend([trg_lang] * 1012)
    # The dev set is 997 sentences, so it should be that times number of languages
    if not len(trg_data) == 1012 * len(trg_langs):
        raise ValueError(f"{len(trg_data)=} != {1012 * len(trg_langs)=}")

# Create the pandas dataframe
df = pd.DataFrame({"source": src_data, "label": trg_data, "trg_lang": trg_lang_data})

In [4]:
def load_hyp_from_tsv(filename: str) -> list[str]:
    with open(filename, "r") as f:
        # Load the third column from the csv reader after the header
        reader = csv.reader(f, delimiter="\t")
        next(reader)
        return [x[2] for x in reader]

# get all the files in the system_outputs folder
experiment_runs = []
for subdir in os.listdir("system_outputs"):
    # The sub_directory must be a directory
    subdir_path = os.path.join("system_outputs", subdir)
    if not os.path.isdir(subdir_path):
        raise ValueError(f"{subdir_path=} is not a directory")
    # get all the files in the sub_directory
    all_files = os.listdir(subdir_path)
    trg_results = []
    for trg_lang in trg_langs:
        # find the tsv file
        trg_file = [f for f in all_files if f.endswith(f"{trg_lang}.tsv")]
        if len(trg_file) == 1:
            trg_results.extend(load_hyp_from_tsv(os.path.join(subdir_path, trg_file[0])))
            continue
        # find the hyp file
        trg_file = [f for f in all_files if f.endswith(f"{trg_lang}-devtest.hyp")]
        if len(trg_file) == 1:
            with open(os.path.join(subdir_path, trg_file[0]), "r") as f:
                trg_results.extend([x.strip() for x in f.readlines()])
            continue
        # Die
        raise ValueError(f"Expected tsv or hyp file with {trg_lang=} but found none in {subdir_path}")
    if not len(trg_results) == 1012 * len(trg_langs):
        raise ValueError(f"{len(trg_results)=} != {1012 * len(trg_langs)=}")
    experiment_run = ExperimentRun(
        name=subdir,
        parameters={"name": subdir},
        predictions=trg_results,
    )
    experiment_runs.append(experiment_run)
    

In [5]:
chrf_evaluator = evaluate.load("chrf")

@distill
def chrf(df, ops: ZenoOptions):
    outputs = df[ops.output_column]
    labels = df[ops.label_column]
    results = [
        chrf_evaluator.compute(predictions=[o], references=[[l]])["score"] for o, l in zip(outputs, labels)
    ]
    return DistillReturn(distill_output=results)

@metric
def avg_chrf(df, ops: ZenoOptions):
    avg = df[ops.distill_columns["chrf"]].mean()
    if pd.isnull(avg) or math.isnan(avg):
        return MetricReturn(metric=0)
    return MetricReturn(metric=avg)

In [ ]:
functions = [
    output_length,
    input_length,
    chrf,
    avg_chrf,
    # bert_score,
    # avg_bert_score,
]

visualize(
    df,
    trg_data,
    experiment_runs,
    "text-classification",
    "source",
    functions,
    zeno_config={"cache_path": "zeno_cache"},
    
)
     

nllb_moe {'name': 'nllb_moe'}
tt-five {'name': 'tt-five'}
tt-zero {'name': 'tt-zero'}
gpt4_tt-five {'name': 'gpt4_tt-five'}



Zeno running on http://localhost:8000
Running predistill functions



preprocessing input_length: 100%|██████████| 1/1 [00:00<00:00, 99.25it/s]


Running inference


Inference on tt-five: 100%|██████████| 1/1 [00:00<00:00, 29.02it/s]

Inference on tt-zero: 100%|██████████| 1/1 [00:00<00:00, 29.72it/s]


Inference on gpt4_tt-five: 100%|██████████| 1/1 [00:00<00:00, 30.66it/s]


...

postprocessing output_length on tt-five: 100%|██████████| 1/1 [00:00<00:00, 60.40it/s]

postprocessing output_length on tt-zero: 100%|██████████| 1/1 [00:00<00:00, 65.71it/s]


postprocessing output_length on gpt4_tt-five: 100%|██████████| 1/1 [00:00<00:00, 63.93it/s]





i=0, o='"Tagad mums ir četru mēnešu vecas pelītes, kurām nav cukura diabēta, bet kurām agrāk bija diabēts", viņš piebilda.'
i=1, o='Dr. Ehuds Urs, Dalhauzijas universitātes medicīnas profesors un Kanādas Diabēta asociācijas klīniskās un zinātniskās nodaļas vadītājs, brīdināja, ka pētījumi vēl tikai sākas.'
i=2, o='Tāpat kā citi speciālisti, viņš ir skeptisks par to, vai diabētu ir iespējams izārstēt, un atzīmē, ka šie atklājumi nav svarīgi cilvēkiem, kam jau ir 1. tipa diabēts.'
i=3, o='Otrdien Zviedrijas akadēmijas Nobela prēmijas Literatūras komitejas pastāvīgā sekretāre Sara Daniusa publicēja paziņojumu Sveriges Radio raidījumā, ka komiteja, nespējot tieši sazināties ar Bobu Dilanu par 2016. gada Nobela prēmijas Literatūrā saņemšanu, ir pārtraukusi centienus ar viņu sazināties.'
i=4, o='Daniuss teica: "Šobrīd mēs neko nedarām. Es esmu piezvanījis un nosūtījis e-pastu viņa tuvākajam sadarbības partnerim un saņēmis ļoti draudzīgas atbildes. Pagaidām tas noteikti ir pietiekami".'
i=5, 

i=0, o='"Tagad mums ir četru mēnešu veci peliņi, kas agrāk bija diabētiski, bet tagad nav," viņš piebilda.'
i=1, o='Dr. Ehud Ur, medicīnas profesors Dalhousie Universitātē Halifaksā, Nova Skotijā un Kanādas Diabēta asociācijas klīniskās un zinātniskās nodaļas vadītājs, brīdināja, ka pētījums vēl ir agrīnā stadijā.'
i=2, o='Kā arī daži citi eksperti, viņš ir skeptisks par to, vai diabētu var izārstēt, norādot, ka šie atklājumi nav nozīmīgi cilvēkiem, kuri jau cieš no 1. tipa diabēta.'
i=3, o='Pirmdien, Sarai Daniusai, Zviedrijas Akadēmijas Nobela literatūras komitejas pastāvīgajai sekretārei, publiski paziņoja radio programmā uz Sveriges Radio Zviedrijā, ka komiteja, nevarot sazināties ar Bobu Dilanu tieši par 2016. gada Nobela literatūras balvas iegūšanu, bija pārtraukusi centienus sazināties ar viņu.'
i=4, o='Danius teica: "Pašlaik mēs neko nedarām. Es esmu zvanījis un nosūtījis e-pastus viņa tuvākajam sadarbības partnerim un saņēmis ļoti draudzīgas atbildes. Pagaidām tas noteikti ir 

i=0, o='"Mums tagad ir četru mēnešu veci peliņi, kas nav diabētiski, bet agrāk bija diabētiski," viņš piebilda.'
i=1, o='Dr. Ehud Ur, medicīnas profesors Dalhousie Universitātē Halifaksā, Nova Skotijā un Kanādas Diabēta asociācijas klīniskās un zinātniskās nodaļas vadītājs, brīdināja, ka pētījums joprojām ir sākuma stadijās.'
i=2, o='Kā arī citi eksperti, viņš ir skeptisks par to, vai diabētu var izārstēt, norādot, ka šie atklājumi nav saistīti ar cilvēkiem, kuri jau cieš no 1. tipa diabēta.'
i=3, o='Pirmdien, Sara Danius, Zviedrijas Akadēmijas Nobelu komitejas literatūras pastāvīgā sekretāre, publiski paziņoja radio programmas "Sveriges Radio" raidījumā Zviedrijā, ka komiteja, nevarot sazināties ar Bobu Dilanu tieši par 2016. gada Nobelu literatūrā, bija pārtraukusi centienus sazināties ar viņu.'
i=4, o='Danius teica: "Pašlaik mēs neko nedarām. Es esmu zvanījis un sūtījis e-pastus viņa tuvākajam sadarbības partnerim un saņēmis ļoti draudzīgas atbildes. Pagaidām tas noteikti ir pietiek

i=0, o='"Mums tagad ir četrus mēnešus vecas peles, kuras vairs nav diabētiķes, bet agrāk bija," viņš piebilda.'
i=1, o='Dr. Ehud Ur, medicīnas profesors Dalhauzijas Universitātē Halifaxā, Novā Skotijā, un Kanādas Diabēta asociācijas klīniskās un zinātniskās nodaļas vadītājs brīdina, ka pētījumi vēl ir sākumstadijā.'
i=2, o='Tāpat kā daži citi eksperti, viņš ir skeptisks par to, vai diabētu var izārstēt, norādot, ka šiem atklājumiem nav nekādas nozīmes cilvēkiem, kuriem jau ir 1. tipa diabēts.'
i=3, o='Pirmdien, Sara Danius, pastāvīgā sekretāre Zviedru Akadēmijas Nobela literatūras komitejā, publiski paziņoja Sveriges Radio raidījumā Zviedrijā, ka komiteja, nespējot tieši sazināties ar Bobu Dilanu par 2016. gada Nobela prēmijas literatūrā iegūšanu, ir pārtraukusi mēģinājumus viņu sasniegt.'
i=4, o='Danius teica: "Pašlaik mēs neko nedaram. Es esmu zvanījis un sūtījis e-pastus viņa tuvākajam sadarbības partnerim un saņēmis ļoti draudzīgas atbildes. Pagaidām tas noteikti ir pietiekami."'
i